# Standard Optimal Power Flow

In [ ]:
import optalg
import pfnet

## Parse network

In [ ]:
parser = pfnet.ParserMAT()
net = parser.parse('../../cases/case1354pegase.mat')

## Construct problem

In [ ]:
net.set_flags('bus',['variable','bounded'],'any','voltage magnitude')
net.set_flags('bus','variable','not slack','voltage angle')
net.set_flags('generator',['variable','bounded'],'any',['active power','reactive power'])

problem = pfnet.Problem(net)
problem.add_function(pfnet.Function('generation cost',1.,net))
problem.add_constraint(pfnet.Constraint('AC power balance',net))
problem.add_constraint(pfnet.Constraint('AC branch flow limits',net))
problem.add_constraint(pfnet.Constraint('variable bounds',net))
problem.analyze()
problem.show()

## Solve problem

In [ ]:
solver = optalg.opt_solver.OptSolverINLP()
solver.set_parameters({'quiet':True})
solver.solve(problem)
print(solver.get_status(), solver.get_iterations())

## Get results

In [ ]:
net.update_properties()
print(net.gen_P_cost, net.bus_P_mis)
net.set_var_values(solver.get_primal_variables())
net.update_properties()
print(net.gen_P_cost, net.bus_P_mis)